In [1]:
import pandas as pd
import json
from sqlalchemy import create_engine

## The Task (Part 2)

Write code (ideally in Python) to import the relevant fields in starlink_historical_data.json as a time series. 
The relevant fields are: - spaceTrack.creation_date (represents the time that the lat/lon records were recorded) - 
longitude - latitude - id (this is the starlink satellite id) Again, the goal is that we want to be able to query 
the database for the last known position for a given starlink satellite. Don't hesitate to use any tools/tricks 
you know to load data quickly and easily!

In [2]:
starlink_historical_data = pd.read_json('../starlink_historical_data.json')

In [3]:
starlink_historical_data_time_series =  starlink_historical_data[['latitude','longitude','id','spaceTrack']]

In [4]:
starlink_hd_ts_extended = pd.concat([starlink_historical_data_time_series[['latitude','longitude','id']],\
pd.DataFrame(starlink_historical_data_time_series['spaceTrack'].values.tolist())], axis=1)

In [5]:
starlink_hd_ts_extended = starlink_hd_ts_extended[['latitude','longitude','id','CREATION_DATE']].\
rename(columns={'CREATION_DATE':'creation_date'})

In [6]:
starlink_hd_ts_extended['creation_date'] = pd.to_datetime(starlink_hd_ts_extended['creation_date'])
starlink_hd_ts_extended['creation_day_of_month'] = starlink_hd_ts_extended['creation_date'].dt.day

In [7]:
starlink_hd_ts_extended = starlink_hd_ts_extended[['id','creation_date','creation_day_of_month','latitude','longitude']].\
rename(columns={'id':'_id','creation_date':'_at','creation_day_of_month':'day_of_month'})

In [8]:
starlink_hd_ts_extended

,_id,_at,day_of_month,latitude,longitude
0,5eed770f096e59000698560d,2020-10-13 04:16:08,13,NaN,86
1,5eed770f096e59000698560e,2020-09-28 19:26:08,28,NaN,48
2,5eed770f096e59000698560f,2020-10-13 17:46:09,13,NaN,62
3,5eed770f096e590006985610,2021-01-26 02:30:00,26,25.453949,109
4,5eed770f096e590006985611,2020-09-02 18:57:38,2,NaN,118
...,...,...,...,...,...
3136,60106f20e900d60006e32cc2,2020-05-19 06:26:10,19,7.145826,164
3137,60106f20e900d60006e32cbd,2020-05-19 06:26:10,19,0.699360,141
3138,60106f20e900d60006e32cc4,2020-05-19 06:26:10,19,0.007645,104
3139,60106f21e900d60006e32cc8,2020-05-19 22:56:10,19,-1.360752,3


In [9]:
user, password, host, port, db, schema = ('postgres','4y7sV96vA9wv46VR','0.0.0.0','5432','postgres','blue_onion')

In [10]:
conn_string = f"postgresql://{user}:{password}@{host}:{port}/{db}"
conn = create_engine(conn_string).connect()
starlink_hd_ts_extended.to_sql('starlink_data_points', con=conn, schema=schema, if_exists='append', index=False)
conn.close()

## The Task (Part 3)

Write logic to fetch/query the last known position of a satellite (by id), given a time T. 
Include this query in your README or somewhere in the project submission

In [69]:
user, password, host, port, db, schema = ('postgres','4y7sV96vA9wv46VR','0.0.0.0','5432','postgres','blue_onion')

In [73]:
def last_known_position(_id,data_time_point):
    stringyfied_timestamp = datetime.strftime(data_time_point,'%Y/%m/%d %H:%M:%S')
    
    conn_string = f"postgresql://{user}:{password}@{host}:{port}/{db}"
    conn = create_engine(conn_string).connect()
    res = conn.execute(
    f"""
    select _id, latitude, longitude
    from blue_onion.starlink_data_points
    where _id = '{_id}' and _at = 
    (
    select max(_at) from blue_onion.starlink_data_points
    where _id = '{_id}' and _at <= '{stringyfied_timestamp}'
    )
    """)
    
    try:
        last_position_row = res.first()
        last_position = (last_position_row[1],last_position_row[2])
        return last_position
    except:
        return (-1,-1)

In [75]:
timestamp_str = '2021-01-26 07:00:00'
_id = '5eed7714096e59000698563e'
data_time_point = datetime.strptime(timestamp_str, '%Y-%m-%d %H:%M:%S')
last_position = last_known_position(_id,data_time_point)

print(f'lets see the last position: {last_position}')

lets see the last position: (-1, -1)


## Bonus Task (Part 4)

Write some logic (via a combination of query + application logic, most likely) to fetch from the 
database the closest satellite at a given time T, and a given a position on a globe as a (latitude, longitude) 
coordinate. No need to derive any fancy math for distances for a point on the globe to a position above the earth. 
You can just use the Haversine formula. Example library to help here https://github.com/mapado/haversine

    
    